In [12]:
import pickle


with open('data/save_test_data_seq.pkl', 'rb') as f:
    data_seq = pickle.load(f)
len_data = len(data_seq)
print(data_seq[0].keys())
with open('data/save_test_data_mean.pkl', 'rb') as f:
    data_mean = pickle.load(f)
with open('data/save_test_data_single.pkl', 'rb') as f:
    data_single = pickle.load(f)

def fn(input):
    for i in range(len_data):
        input[i]['ref_sentences'] = data_seq[i]['ref_sentences']
        assert input[i]['video_name'] == data_seq[i]['video_name']
    
fn(data_mean)
fn(data_single)

# print(data[100])
for i in range(3):
    print(len(data_seq[i]['pred_sentence']), len(data_seq[i]['ref_sentences']))
    # print(data[i]['pred_sentence'])
    # print(data[i]['ref_sentences'])

dict_keys(['video_name', 'video_id', 'pred_sentence', 'ref_sentences'])
18 52
29 54
30 62


In [16]:
import tqdm
import itertools
from evaluation.cider import Cider
from evaluation.meteor import Meteor
from evaluation.bleu import Bleu
from evaluation.rouge import Rouge

import numpy as np
from evaluation.tokenizer import PTBTokenizer

data = data_seq

def compute_scores(gts, gen):
    metrics = (Cider(), Meteor(), Bleu(), Rouge()) 
    all_score = {}
    all_scores = {}
    for metric in metrics:
        score, scores = metric.compute_score(gts, gen)
        all_score[str(metric)] = score
        all_scores[str(metric)] = scores

    n_bleu = len(all_score['BLEU'])
    bleu_scores = all_score['BLEU']
    all_score['BLEU'] = np.exp ( np.mean(np.log(bleu_scores)) )
    return all_score, all_scores

gen, gts = {}, {}

len_data = len(data)
for i in range(len_data):
    gts_i, gen_i =  data[i]['ref_sentences'], data[i]['pred_sentence'] # gts_i: list, gen_i: str
    video_id = data[i]['video_id']

    # gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
    # print(gen_i)
    gen[i] = [gen_i, ]
    gts[i] = gts_i


gts = PTBTokenizer.tokenize(gts)
gen = PTBTokenizer.tokenize(gen)
# print(gen[0])
# print(gts[0])
scores, _ = compute_scores(gts, gen)

print(scores)


{'CIDEr': 0.549278193725268, 'METEOR': 0.2970440063021677, 'BLEU': 0.5065889189044042, 'ROUGE': 0.6410289388579278}


In [ ]:
# for i in range(3):
#     gts_i, gen_i =  data[i]['ref_sentences'], data[i]['pred_sentence']
#     video_id = data[i]['video_id']
    
#     # for i, (gts_i, gen_i) in enumerate(zip(caps_gt, caps_gen)):
#     gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
#     gen[video_id] = [gen_i, ]
#     gts[video_id] = gts_i

#     cider_fn = Cider()
#     cider_fn.compute_score(gen, gts)
#     # PTBTokenizer.tokenize(data[0]['pred_sentence'])

In [ ]:



def evaluate_metrics(model, dataloader, text_field, exp_name, epoch):
    import itertools

    model.eval()

    gen = {}
    gts = {}
    with tqdm(desc='Epoch %d - evaluation' % e, unit='it', total=len(dataloader)) as pbar:
        for it, (images, caps_gt) in enumerate(iter(dataloader)):

            images = images.to(device)

            with torch.no_grad():
                out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<|endoftext|>'], 5, out_size=1)
            caps_gen = text_field.decode(out, join_words=False)
            for i, (gts_i, gen_i) in enumerate(zip(caps_gt, caps_gen)):
                gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
                gen['%d_%d' % (it, i)] = [gen_i, ]
                gts['%d_%d' % (it, i)] = gts_i
            pbar.update()

    gts = evaluation.PTBTokenizer.tokenize(gts)
    gen = evaluation.PTBTokenizer.tokenize(gen)



    scores, _ = evaluation.compute_scores(gts, gen)
    return scores


In [11]:
from torchtext.data.metrics import bleu_score

def evaluation(y_pred, y_true, max_n):
    """
    Args:
        y_true: shape (B, len_seq) an iterable of candidate translations. Each translation is an iterable of tokens,
        e.g., [['My', 'full', 'pytorch', 'test'], ['Another', 'Sentence']];
        
        y_pred: prediction, in same format as y_true;

        max_n: the maximum n-gram we want to use. E.g. if max_n=3, we will use unigrams, bigrams and trigram;

    Return:
        BLEU  METEOR  CIDEr   SPICE
    """
    bleu_weights = [ 1 / max_n for _ in range(max_n) ]
    bleu = bleu_score(y_pred, y_true, max_n=max_n, weights=bleu_weights)

    return bleu

    
    meteor = Meteor()
    meteor = meteor.compute_score(y_true, y_pred)

    
    return bleu, meteor
    


In [27]:
candidate_corpus = [['My', 'full', 'pytorch', 'test'], ['Another', 'Sentence']]
candidate_corpus = [['My', 'big', 'pytorch', 'test'], ['Another', 'Sentence']]

references_corpus = [[['My', 'full', 'pytorch', 'test'], ['Completely', 'Different']], [['No', 'Match']]]


a = evaluation(candidate_corpus, references_corpus, 2)
print(a)

0.0


In [7]:
import os
import subprocess
import threading
import tarfile

import requests

def download_from_url(url, path):
    """Download file, with logic (from tensor2tensor) for Google Drive"""
    if 'drive.google.com' not in url:
        print('Downloading %s; may take a few minutes' % url)
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        with open(path, "wb") as file:
            file.write(r.content)
        return
    print('Downloading from Google Drive; may take a few minutes')
    confirm_token = None
    session = requests.Session()
    response = session.get(url, stream=True)
    for k, v in response.cookies.items():
        if k.startswith("download_warning"):
            confirm_token = v

    if confirm_token:
        url = url + "&confirm=" + confirm_token
        response = session.get(url, stream=True)

    chunk_size = 16 * 1024
    with open(path, "wb") as f:
        for chunk in response.iter_content(chunk_size):
            if chunk:
                f.write(chunk)


                

METEOR_GZ_URL = 'http://aimagelab.ing.unimore.it/speaksee/data/meteor.tgz'
METEOR_JAR = 'meteor-1.5.jar'

class Meteor:
    def __init__(self):
        __file__ = '/users/yluo73/workspace/cv1430/DL2470/nlp_metrics.ipynb'
        base_path = os.path.dirname(os.path.abspath(__file__))
        jar_path = os.path.join(base_path, METEOR_JAR)
        gz_path = os.path.join(base_path, os.path.basename(METEOR_GZ_URL))
        if not os.path.isfile(jar_path):
            if not os.path.isfile(gz_path):
                download_from_url(METEOR_GZ_URL, gz_path)
            tar = tarfile.open(gz_path, "r")
            tar.extractall(path=os.path.dirname(os.path.abspath(__file__)))
            tar.close()
            os.remove(gz_path)

        self.meteor_cmd = ['java', '-jar', '-Xmx2G', METEOR_JAR, \
                '-', '-', '-stdio', '-l', 'en', '-norm']
        self.meteor_p = subprocess.Popen(self.meteor_cmd, \
                cwd=os.path.dirname(os.path.abspath(__file__)), \
                stdin=subprocess.PIPE, \
                stdout=subprocess.PIPE, \
                stderr=subprocess.PIPE)
        # Used to guarantee thread safety
        self.lock = threading.Lock()

    def compute_score(self, gts, res):
        assert(gts.keys() == res.keys())
        imgIds = gts.keys()
        scores = []

        eval_line = 'EVAL'
        self.lock.acquire()
        for i in imgIds:
            assert(len(res[i]) == 1)
            stat = self._stat(res[i][0], gts[i])
            eval_line += ' ||| {}'.format(stat)

        self.meteor_p.stdin.write('{}\n'.format(eval_line).encode())
        self.meteor_p.stdin.flush()
        for i in range(0,len(imgIds)):
            scores.append(float(self.meteor_p.stdout.readline().strip()))
        score = float(self.meteor_p.stdout.readline().strip())
        self.lock.release()

        return score, scores

    def _stat(self, hypothesis_str, reference_list):
        # SCORE ||| reference 1 words ||| reference n words ||| hypothesis words
        hypothesis_str = hypothesis_str.replace('|||','').replace('  ',' ')
        score_line = ' ||| '.join(('SCORE', ' ||| '.join(reference_list), hypothesis_str))
        self.meteor_p.stdin.write('{}\n'.format(score_line).encode())
        self.meteor_p.stdin.flush()
        raw = self.meteor_p.stdout.readline().decode().strip()
        numbers = [str(int(float(n))) for n in raw.split()]
        return ' '.join(numbers)

    def __del__(self):
        self.lock.acquire()
        self.meteor_p.stdin.close()
        self.meteor_p.kill()
        self.meteor_p.wait()
        self.lock.release()

    def __str__(self):
        return 'METEOR'